### File Explanations
- Kakao NLI Datasets
  - snli Dataset(Big, 500K) : Train, Test와 겹치는지 확인, 1개 겹치는 Hypothesis가 있어 제거함
  - xnli.dev Dataset(Small, 2K) : 겹치는 항목 없음
  - Data Available at : [KakaoBrain Github](https://github.com/kakaobrain/KorNLUDatasets/tree/master/KorNLI)

In [63]:
from tqdm import tqdm

In [64]:
import pandas as pd

TRAIN = '/content/drive/MyDrive/DACON_MONTHLYNLI/train_data.csv'
TEST = '/content/drive/MyDrive/DACON_MONTHLYNLI/test_data.csv'
NEW = '/content/drive/MyDrive/DACON_MONTHLYNLI/snli.tsv'
train = pd.read_csv(TRAIN)
test = pd.read_csv(TEST)
new = pd.read_csv(NEW, sep = '\t')

In [65]:
new.columns = ['premise', 'hypothesis', 'label']
new

,premise,hypothesis,label
0,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 경쟁을 위해 말을 훈련시키고 있다.,neutral
1,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 식당에서 오믈렛을 주문하고 있다.,contradiction
2,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,사람은 야외에서 말을 타고 있다.,entailment
3,카메라에 웃고 손을 흔드는 아이들,그들은 부모님을 보고 웃고 있다,neutral
4,카메라에 웃고 손을 흔드는 아이들,아이들이 있다,entailment
...,...,...,...
550147,네 명의 더러운 맨발의 아이들.,4명의 아이들이 '가장 깨끗한 발'로 상을 받았다,contradiction
550148,네 명의 더러운 맨발의 아이들.,네 명의 노숙자 아이들이 신발을 도둑맞아서 그들의 발이 더러워졌다.,neutral
550149,한 남자가 아름다운 푸른 물에서 바디슈트를 입고 서핑을 하고 있다.,바디슈트를 입은 남자가 서핑 대회에 참가하고 있다.,neutral
550150,한 남자가 아름다운 푸른 물에서 바디슈트를 입고 서핑을 하고 있다.,비즈니스 슈트를 입은 남자가 이사회로 향하고 있다.,contradiction


In [67]:
premise = train['premise'].tolist()
hypothesis = train['hypothesis'].tolist()
check = 0
for idx in tqdm(range(len(new))):
  query1 = new.iloc[idx]['premise']
  query2 = new.iloc[idx]['hypothesis']
  if query1 in premise:
    check += 1
    print(f"{idx} - item in new dataset[premise] is discovered in train dataset[premise]")
  if query1 in hypothesis:
    check += 1
    print(f"{idx} - item is new dataset[premise] is discovered in train dataset[hypothesis]")
  if query2  in premise:
    check += 1
    print(f"{idx} - item in new dataset[hypothesis] is discovered in train datset[premise]")
  if query2 in hypothesis:
    check +=1
    print(f"{idx} - item in new dataset[hypothesis] is discovered in train datset[hypothesis]")

if check == 0:
  print("No matches with train dataset : no data leakage")

 91%|█████████ | 500061/550152 [17:19<01:40, 496.70it/s]

499986 - item in new dataset[hypothesis] is discovered in test datset[hypothesis]


100%|██████████| 550152/550152 [19:02<00:00, 481.64it/s]


In [68]:
premise = test['premise'].tolist()
hypothesis = test['hypothesis'].tolist()
check = 0
for idx in tqdm(range(len(new))):
  query1 = new.iloc[idx]['premise']
  query2 = new.iloc[idx]['hypothesis']
  if query1 in premise:
    check += 1
    print(f"{idx} - item in new dataset[premise] is discovered in test dataset[premise]")
  if query1 in hypothesis:
    check += 1
    print(f"{idx} - item is new dataset[premise] is discovered in test dataset[hypothesis]")
  if query2  in premise:
    check += 1
    print(f"{idx} - item in new dataset[hypothesis] is discovered in test datset[premise]")
  if query2 in hypothesis:
    check +=1
    print(f"{idx} - item in new dataset[hypothesis] is discovered in test datset[hypothesis]")

if check == 0:
  print("No matches with test dataset : no data leakage")

100%|██████████| 550152/550152 [02:25<00:00, 3775.99it/s]

No matches with test dataset : no data leakage


In [69]:
new.iloc[499986]['hypothesis']

'방이 깨끗하다.'

In [73]:
train['hypothesis'].tolist().index(new.iloc[499986]['hypothesis'])

1939

In [75]:
print(train.iloc[1939])
print(new.iloc[499986])

index                             1939
premise       방도 사진과 마찬가지로 깨끗하고 훌륭합니다.
hypothesis                    방이 깨끗하다.
label                       entailment
Name: 1939, dtype: object
premise       한 여자가 벽이 그을린 지저분한 방의 책상에 서 있다.
hypothesis                          방이 깨끗하다.
label                          contradiction
Name: 499986, dtype: object


In [76]:
new = new.drop(499986)
new.iloc[499986]

premise       한 여자가 벽이 그을린 지저분한 방의 책상에 서 있다.
hypothesis                여자는 엉망진창인 집을 조사한다.
label                                neutral
Name: 499987, dtype: object

In [77]:
train = pd.concat([train, new])

In [80]:
train['premise'] = train['premise'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9]', '')
train['hypothesis'] = train['hypothesis'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9]', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [83]:
train = train.drop('index', axis = 1)

In [86]:
train.to_csv('new_big_train.csv')